In [313]:
!pip install spaczz
!pip install apyori
!pip install mlxtend
!pip install textblob
!pip install transformers torch sentencepiece
!pip install scikit-learn


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [316]:
import os
import re
import copy
import pandas as pd
import time
import string
import itertools

from mlxtend.frequent_patterns import apriori, association_rules
from textblob import TextBlob
import copy

import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer

from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch.nn.functional as F
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore")

nltk.download("wordnet")
nltk.download("omw-1.4")
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /Users/tomhill/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/tomhill/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /Users/tomhill/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/tomhill/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/tomhill/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tomhill/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# 1. Product Feature Extraction

## Crawl Reviews
The first part of this system is Crawl Reviews and is adapted from Hu & Liu (n.d). This initial step extracts the reviews from the .txt files in the `/Data` folder and then builds the foundations of the Review Database, which in this instance is the `df_product_reviews` dataframe in the following step. Each line is extracted into a Pandas dataframe (Python, n.d). It's done in this way to keep the original raw product lines in a separate dataframe. Pandas dataframes are particularly useful since they make storing and extracting data a lot easier than, for example, spreadsheets (Python, n.d). Also, since this system is written in Python, many of the other libraries we will be using expect a Pandas dataframe such as the apriori algorithm used in the `Frequent Features Identification` step (Medium, 2024).

Each line in the .txt files are either a review or the title of a review - both of which contain product feature sentiments. It is important to note that there have been changes made by the author to some of the original .txt files such as removing squares of <strong>*</strong> which denoted a header in some of the product files, as well as removing some README files. Both of these were removed since they did not have product reviews in them which meant they had no bearing on classifying product features in future steps.

In [423]:
def read_all_files_in_folder(directory):
    dataframes = []
    for root, directories, files in os.walk(directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            df = pd.read_table(file_path, on_bad_lines='skip', encoding='cp1252', header=None)
            product_name = filename.replace('.txt', '')
            df['product'] = product_name
            dataframes.append(df)
    
    combined_df = pd.concat(dataframes, ignore_index=True)
    combined_df.columns = ['line', 'product'] 
    combined_df = combined_df[['product', 'line']]
    
    return combined_df

df_raw_product_lines = read_all_files_in_folder("./data")
df_raw_product_lines.describe()

,product,line
count,10920,10920
unique,17,10531
top,Creative Labs Nomad Jukebox Zen Xtra 40GB,[t]
freq,1811,325


## Parse Reviews
In each line, there are currently tags such as <strong>[t]</strong>, <strong>feature[+/-]</strong> and <strong>##</strong>. For example:

In [335]:
df_raw_product_lines.head()

,product,line
0,Nokia 6610,"[t]excellent phone , excellent service ."
1,Nokia 6610,##i am a business user who heavily depend on m...
2,Nokia 6610,"phone[+3], work[+2]##there is much which has b..."
3,Nokia 6610,##just double check with customer service to e...
4,Nokia 6610,at&t customer service[-2]##after several years...


According to the README files that were removed, we know that <strong>[t]</strong> at the beginning of a line denotes a title. We do not need to see the annotation itself, however, because this does not help us find product features downstream in the preprocessing pipeline. The text following it, however, does. Furthermore, we know from the README files that <strong>feature[+/-]</strong> denotes groundtruth labels of the review sentences provided by the authors Hu & Liu (n,d). We want to keep these along with the review sentences, denoted by <strong>##</strong> since these will be used to analyse how well our classification models performed in our sentiment analysis in the future. Finally, we obviously want to keep the text following <strong>##</strong> since these will contain sentiments of particular product features.

As you can see, these are all parsed out using regex on each line in the reviews. If there are annotations, then these are stored with the review sentences. This will make it easier to analyse how well the classification models have performed because having the groundtruth labels in line with the review sentences will mean that we can compare how a human interpreted feature sentiments against our ML models and therefore approximate how accurate they are (Understand.ai, n.d).

In [343]:
def parse_reviews(df):
    df['annotated_labels'] = df['line'].apply(lambda x: ' '.join(re.findall(r'(.*)##', x)))
    df['review'] = df['line'].apply(lambda x: ' '.join(re.findall(r'##(.*)', x)))
    df['review'] = df.apply(
        lambda row: ' '.join(re.findall(r'\[t\](.*)', row['line'])) if row['review'] == '' else row['review'],
        axis=1
    )
    return df

df_product_reviews = parse_reviews(df_raw_product_lines)
df_product_reviews[['product', 'line', 'annotated_labels', 'review']]

,product,line,annotated_labels,review
0,Nokia 6610,"[t]excellent phone , excellent service .",,"excellent phone , excellent service ."
1,Nokia 6610,##i am a business user who heavily depend on m...,,i am a business user who heavily depend on mob...
2,Nokia 6610,"phone[+3], work[+2]##there is much which has b...","phone[+3], work[+2]",there is much which has been said in other rev...
3,Nokia 6610,##just double check with customer service to e...,,just double check with customer service to ens...
4,Nokia 6610,at&t customer service[-2]##after several years...,at&t customer service[-2],after several years of torture in the hands of...
...,...,...,...,...
10915,Canon PowerShot SD500,automatic shutter[-3]##There were many times w...,automatic shutter[-3],There were many times when I clicked the autom...
10916,Canon PowerShot SD500,##It seems you have to double click some of th...,,It seems you have to double click some of the ...
10917,Canon PowerShot SD500,##Huh? I missed out on lots of photos and lots...,,Huh? I missed out on lots of photos and lots a...
10918,Canon PowerShot SD500,controls[-1]##The controls are hard to manipul...,controls[-1],"The controls are hard to manipulate, especiall..."


### Preprocess
Preprocessing data is crucial for removing noisy information, enriching information and cleaning data before running it through other algorithms (such as the classification algorithms at the end of this report)(Singh et al, 2021). The preprocessing steps here are influenced by the preprocessing steps in Hu & Liu (n.d). These preprocessing steps are required before the feature identification steps in the downstream parts of the pipeline because they ensure that the data is in a consistent format, reduce dimensionality, and remove irrelevant or redundant features (Singh et al, 2021). This should ultimately improve the accuracy and efficiency of the subsequent feature identification and classification processes although there was no comparative assesment done on this particular section so further work could validate the effectiveness of these preprocessing steps on the overall performance of the pipeline.

The preprocessing steps are  deliberately performed in a particular order, the justifications of which are as follows:
1. Word tokenisation: Word tokenisation breaks a sentence into individual tokens so it is easier to perform future preprocessing steps like remove stopwords, remove punctuation and lemmatization (Zakaria, 2024).
2. Remove stopwords and punctuation: stopwords and punctuation are often useless for determining product features and their sentiment, so they are removed (Hu & Liu, n.d). This is deliberately done prior to PoS tagging to reduce the noise in the text, making it easier for the PoS tagger to identify relevant words accurately. This step is also deliberately done before lemmatization because lemmatization is computationally expensive so performing this step first reduces the computational overheard required for lemmatization (Jain, 2024). The algorithm used `str.maketrans` which is to remove the string if it is just quotation marks. An empty string does not hold meaniningful information to help us classify sentences.
3. Lowercase words: words are lowercased to maintain consistency for each word. Words are lowercased after removing stopwords since some words that are not stopwords can become those when lowercased (such as US the country becoming us, which is a stopword). It is also done before PoS tagging and lemmatization to ensure consistency and to avoid discrepancies caused by case differences (Ved, 2020).
4. Parts of Speech (PoS) Tagging: this labels all PoS with the correct annotations. This is important for adjectives, since these are potential sentiments, and nouns, since these are potential product features (Hu & Liu, n.d). These tags are then used to analyse adjectives and nouns in the relevant stages later in the pipelines such as extracting Frequent Features and Infrequent Features with their Opinion Words (Saumyab271, 2022). It is done before lemmatization since it is important that the correct lemma is used, and without the PoS tag, the PoS tagging library can make mistakes and provide the wrong lemma for the token.
5. Lemmatization: this finds the base form, or lemma, of each word and therefore makes it much easier to analyse the usage of different words more easily in later stages (Saumyab271, 2022). For example, if someone in their review stated that “the battery is great” and another stated that “the batteries are excellent”, then lemmatization would lemmatize batteries to battery This would mean that the opinion miner system could then find a common feature, battery, more easily.


In [344]:
class Preprocess:
    def __init__(self):
        self.stop_words = set(nltk.corpus.stopwords.words('english'))
        
    def main(self, reviews):
        tokenize_reviews = self.lower_tokenize_reviews(reviews)
        remove_punctuation_reviews = self.remove_punctuation_reviews(tokenize_reviews)
        remove_empty_strings_reviews = self.remove_empty_strings_reviews(remove_punctuation_reviews)
        remove_stopwords_reviews = self.remove_stopwords_reviews(remove_empty_strings_reviews)
        pos_tag_reviews = self.pos_tag_reviews(remove_stopwords_reviews)
        lemmatize_reviews = self.lemmatize_reviews(pos_tag_reviews)

        return lemmatize_reviews
        
    def lower_tokenize_reviews(self, reviews):
        return reviews.apply(lambda x: nltk.word_tokenize(x.lower()))
        
    def remove_punctuation_reviews(self, reviews):
        return reviews.apply(lambda tokens: [
            word.translate(str.maketrans('', '', string.punctuation))
            for word in tokens
        ])
        
    def remove_empty_strings_reviews(self, reviews):
        return reviews.apply(lambda review: [word for word in review if word])
                             
    def remove_stopwords_reviews(self, reviews):
        return reviews.apply(lambda tokens: [word for word in tokens if word not in self.stop_words])

    def pos_tag_reviews(self, reviews):
        return reviews.apply(lambda x: nltk.pos_tag(x))
        
    def get_wordnet_pos(self, tag):
        if tag.startswith('J'):
            return wordnet.ADJ
        elif tag.startswith('V'):
            return wordnet.VERB
        elif tag.startswith('N'):
            return wordnet.NOUN
        elif tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN

    def lemmatize_reviews(self, reviews):
        wnl = WordNetLemmatizer()
        
        def lemmatize_token(token):
            if isinstance(token, tuple) and len(token) == 2:
                word, tag = token
                return (wnl.lemmatize(word, self.get_wordnet_pos(tag)), tag)
                
        return reviews.apply(lambda tokens: [lemmatize_token(token) for token in tokens])
        

In [345]:
df_product_reviews["review_preprocessed"] = Preprocess().main(df_product_reviews["review"])

df_product_reviews[["product", "review", "review_preprocessed"]]

,product,review,review_preprocessed
0,Nokia 6610,"excellent phone , excellent service .","[(excellent, JJ), (phone, NN), (excellent, JJ)..."
1,Nokia 6610,i am a business user who heavily depend on mob...,"[(business, NN), (user, VBD), (heavily, RB), (..."
2,Nokia 6610,there is much which has been said in other rev...,"[(much, RB), (say, VBD), (reviews, JJ), (featu..."
3,Nokia 6610,just double check with customer service to ens...,"[(double, RB), (check, VB), (customer, NN), (s..."
4,Nokia 6610,after several years of torture in the hands of...,"[(several, JJ), (year, NNS), (torture, NN), (h..."
...,...,...,...
10915,Canon PowerShot SD500,There were many times when I clicked the autom...,"[(many, JJ), (time, NNS), (click, VBD), (autom..."
10916,Canon PowerShot SD500,It seems you have to double click some of the ...,"[(seem, VBZ), (double, JJ), (click, JJ), (time..."
10917,Canon PowerShot SD500,Huh? I missed out on lots of photos and lots a...,"[(huh, NNS), (miss, VBD), (lot, NNS), (photo, ..."
10918,Canon PowerShot SD500,"The controls are hard to manipulate, especiall...","[(control, NNS), (hard, JJ), (manipulate, NN),..."


## Frequent Feature Identification

This step retrieves the frequent features for the opinion miner. Here, association mining is used, and it is done so using the Apriori algorithm (Hu & Liu, n.d). Association mining is a technique used in data mining to find interesting and useful associations between different itemsets (D, 2018). In this opinion miner system, we want to find frequent features for the different products, which means we want to find the common nouns and noun groups that are seen together for each product (Hu & Liu, n.d). Association mining is therefore a useful technique to deploy because we can find which common nouns and noun groups are often seen and therefore, potentially, what some of the frequent features are. Without the previous step tagging tokens with their PoS tags, this would not be possible since we would not know whether a token was a noun or noun group.

Association rule mining uses a minimum support value to state that if a token is associated with another word that is equal to or above the minimum threshold then it is identified as a common itemset and kept (Hu & Liu, n.d). It is deliberately set to a very low value of 0.01 since this will return as many common itemsets as possible that can then be pruned in the Redundancy Pruning and Compactness Pruning steps that follow (Hu & Liu, n.d).

As adapted from Hu & Liu(n.d), Association rule mining works by:
1. defining a set of items, called <strong>I</strong> which are the noun and noun groups that we want to find associate with one another.
2. defining another set of items, called <strong>D</strong>, which is just a subset of items from <strong>I</strong>. <strong>D</strong> and <strong>I</strong> are comapred to identify patterns and relationships between these items based on their co-occurrence within transactions

An item is said to have an association rule if it firstly meets the minimum support threshold, and it also is reliably found with another item in another set (Hu & Liu, n.d). For example, if an item, lets called it <strong>A</strong> is in set <strong>I</strong> and it is seen enough times with an item in set <strong>D</strong>, lets call this item <strong>B</strong>, then <strong>A</strong> is declared to have a sufficiently strong enough relationship with <strong>B</strong>. This means that the itemset `{A, B}` is declared to have a strong enough association rule and indicates a strong enough relationship between <strong>A</strong> and <strong>B</strong>, which means the itemset `{A, B}` is identified as a frequent feature in this step.

In order to prepare the data for association mining we need to extract the noun and noun groups for processing, which we do by using the `extract_noun_and_noun_groups` method below. The `extract_noun_and_noun_groups` method returns the words that are marked with the pos tags `NN` or `NG` since these refer to nouns and noun groups respectively. After this the Apriori algorithm is deployed to find frequent itemsets from the nouns and noun groups (Hu & Liu, n.d). The Apriori algorithm usually performs two steps, however Hu & Liu (n.d) only performed only the first step in the algorithm. This is what has been deployed below as it was shown to be effective (Hu & Liu, n.d). This first step merely scans through the transaction set, which is the set of <strong>D</strong> (a subset of <strong>I</strong>), and finds any of the itemsets that occur frequently together (for example `{A, B}`), as long as they meet the minimum support threshold (Hu & Liu, n.d). After this, any itemsets which contain more than three items are pruned since they were found to be not useful (Hu & Liu, n.d).

In [346]:
def extract_noun_and_noun_groups(pos_tagged_reviews):
    return [[word for word, pos in review if pos.startswith('NN') or pos.startswith('NG')] for review in pos_tagged_reviews]

original_frequent_features_dict = {}

def find_frequent_features():
    for product_name, group in df_product_reviews.groupby('product'):
        reviews_df = group['review_preprocessed'].tolist()
        reviews = extract_noun_and_noun_groups(reviews_df)
        
        te = TransactionEncoder()
        te_ary = te.fit(reviews).transform(reviews)
        df = pd.DataFrame(te_ary, columns=te.columns_)
        
        frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)
        frequent_itemsets_with_three_or_less = frequent_itemsets[frequent_itemsets['itemsets'].apply(lambda x: len(x) <= 3)]
        
        original_frequent_features_dict[product_name] = frequent_itemsets_with_three_or_less

find_frequent_features()

original_frequent_features_dict

{'Apex AD2600 Progressive-scan DVD player':      support                itemsets
 0   0.020262                (ad2600)
 1   0.019070                (amazon)
 2   0.057211                  (apex)
 3   0.015495                 (brand)
 4   0.025030                (button)
 5   0.010727                   (buy)
 6   0.013111                    (cd)
 7   0.023838             (christmas)
 8   0.010727               (control)
 9   0.027414              (customer)
 10  0.021454                   (day)
 11  0.026222                  (disc)
 12  0.011919                  (disk)
 13  0.014303               (display)
 14  0.181168                   (dvd)
 15  0.019070            (everything)
 16  0.036949               (feature)
 17  0.011919                (format)
 18  0.017878                   (get)
 19  0.014303                  (gift)
 20  0.011919                  (hour)
 21  0.011919                   (lot)
 22  0.020262               (machine)
 23  0.014303                 (model)
 24  0.

### Room for future work

There is arguably room for future work here that can both manipulate the minimum support threshold and the number of itemsets pruned in order to see whether better accuracy can be achieved in the classification steps below. There is also an argument whereby another method, such as Pointwise Mutual Information could be used instead of association rule mining to find common itemsets, as this may produce more effective results (Bhalla, n.d)

One potential issue with using the Apriori algorithm is it can be slow on large datasets (GeeksforGeeks, 2018). With the size of the datasets used in this system, however, the time it takes to execute only 0.1 seconds over 10 runs so it is quick.

In [75]:
num_runs = 10
total_time = 0

for _ in range(num_runs):
    start_time = time.time()
    find_frequent_features()
    end_time = time.time()

    total_time += (end_time - start_time)

average_time = total_time / num_runs

print(f"Average execution time over {num_runs} runs: {average_time:.2f} seconds")

Average execution time over 10 runs: 0.10 seconds


## Compactness Pruning

The next section of the pipeline uses the Compactness Pruning algorithm to prune any features found in the Frequent Features which are more than one item in length and appear more than three times together in the same product reviews (Hu & Liu, n.d).

The algorithm works as follows, and is adapted from a description of it in Hu & Liu (n.d):
1. Let `f` be a frequent feature phrase (two or more words that were found in the Frequent Features step), and `s` be a review or title sentence with `w` words in it.
2. Then for each `s` in the preprocessed reviews, loop through those and for each check whether the first word in `f` appears.
3. If it does, then check whether the next word in `f` appears within 3 words of the current word.
4. If it does, then we say that that feature is compact in `s`.
5. If the same f then appears in one more `s` then this `f` is considered a compact feature and retained, otherwise it is pruned.

In [347]:
compactness_pruned_features_dict = {}
min_p_support_value_compactness_pruning = 3

for product_name, frequent_itemsets in original_frequent_features_dict.items():
    frequent_items = frequent_itemsets['itemsets'].tolist()
    frequent_compact_items = defaultdict(lambda: 0)

    # get all of the series for the given product
    group = df_product_reviews[df_product_reviews['product'] == product_name]
    # get the unpreprocessed reviews and split it so that each token is individually in its own index
    reviews = [review.split() for review in group['review'].tolist()]

    # then for each review, look for any of the frequent items
    for ind, review in enumerate(reviews):
        for items in frequent_items:
            index_of_items = []
            for item in items:
                if item in review:
                    index_of_items.append(review.index(item))
            # if all the items in the frequent items were found in a review and they were close to each other
            # then count them as appearing there
            if len(index_of_items) == len(items) and max(index_of_items) - min(index_of_items) < 3:
                frequent_compact_items[items] += 1
    
    # prune the items that appear less frequently
    result_items = [item for item, val in frequent_compact_items.items() if val >= min_p_support_value_compactness_pruning]
    
    # update the frequent_itemsets dataframe and keep only the unpruned itemsets
    pruned_itemsets = frequent_itemsets[frequent_itemsets['itemsets'].isin(result_items)]
    
    # store the pruned itemsets dataframe in a separate dictionary
    compactness_pruned_features_dict[product_name] = pruned_itemsets

Comparing the itemsets we can see a number of items were pruned due to this step:

In [348]:
def display_original_pruned_differences(original_dictionary, pruned_dictionary):
    results = []
    
    for product_name in pruned_dictionary.keys():
        length_pruned = len(pruned_dictionary[product_name])
        length_original_frequent = len(original_dictionary[product_name])
        percentage_reduction = ((length_original_frequent - length_pruned) / length_original_frequent) * 100
        
        results.append({
            'Product': product_name,
            'Original Length': length_original_frequent,
            'Pruned Length': length_pruned,
            'Percentage Reduction (%)': percentage_reduction
        })

    return pd.DataFrame(results).sort_values(by='Percentage Reduction (%)', ascending=False)

In [110]:
display_original_pruned_differences(original_frequent_features_dict, compactness_pruned_features_dict)

,Product,Original Length,Pruned Length,Percentage Reduction (%)
2,Canon PowerShot SD500,120,37,69.166667
6,Diaper Champ,87,41,52.873563
3,Canon S100,91,43,52.747253
7,Hitachi router,87,44,49.425287
16,norton,71,41,42.253521
15,ipod,94,55,41.489362
12,Nokia 6610,78,50,35.897436
10,Nikon coolpix 4300,104,69,33.653846
11,Nokia 6600,74,51,31.081081
8,Linksys Router,82,61,25.609756


### Future work
It is therefore possible to see that some features have been pruned significantly (for example the Canon PowerShot SD500 had over 69% of its features pruned).

It would be valuable to explore whether changing the p-support value could result in more significant increases in accuracy in the final classification models.

## Redundancy Pruning

The next step is Redundancy Pruning (Hu & Liu, n.d). The purpose of this algorithm is to reduce the number of features with just a single word identified in the Frequent Features step (Hu & Liu, n.d). This is done since many of the features identified in the Frequent Features step will be irrelevant due to the low minimum support threshold set in the Apriori algorithm (Hu & Liu, n.d).

Redundancy Pruning initially finds the p-support value for each feature found in the Frequent Features step, which is just a count of the number of times a feature appears in the reviews and titles. A minimum p-support threshold is then set to prune features that fall below the threshold. Then, for each feature that is just a single word - check its p-support value and if it is below the threshold and is a subset of a feature with two or more words in it then it is pruned.
.


In [349]:
min_p_support_threshold = 3
redundancy_pruned_compactness_features_dict = {}

for product_name, frequent_itemsets in compactness_pruned_features_dict.items():
    p_support = defaultdict(lambda: 0)
    # get all the reviews for the product
    reviews = df_product_reviews[df_product_reviews['product'] == product_name]['line']

    for itemset in frequent_itemsets['itemsets']:
        count = reviews.apply(lambda x: all(word in x for word in itemset)).sum()
        p_support[itemset] = count
    
    pruned_itemsets = []
    multi_word_itemsets = [itemset for itemset in p_support if len(itemset) > 1]
    
    for itemset in p_support:
        if len(itemset) == 1: 
            is_subset = any(itemset.issubset(multi_word_itemset) for multi_word_itemset in multi_word_itemsets)
            if p_support[itemset] < min_p_support_threshold and is_subset:
                continue
        pruned_itemsets.append(itemset)
    
    pruned_itemsets_df = frequent_itemsets[frequent_itemsets['itemsets'].isin(pruned_itemsets)]
    redundancy_pruned_compactness_features_dict[product_name] = pruned_itemsets_df

redundancy_pruned_compactness_features_dict

{'Apex AD2600 Progressive-scan DVD player':      support             itemsets
 0   0.020262             (ad2600)
 1   0.019070             (amazon)
 2   0.057211               (apex)
 3   0.015495              (brand)
 4   0.025030             (button)
 5   0.010727                (buy)
 6   0.013111                 (cd)
 7   0.023838          (christmas)
 8   0.010727            (control)
 9   0.027414           (customer)
 10  0.021454                (day)
 11  0.026222               (disc)
 12  0.011919               (disk)
 13  0.014303            (display)
 14  0.181168                (dvd)
 15  0.019070         (everything)
 16  0.036949            (feature)
 17  0.011919             (format)
 18  0.017878                (get)
 19  0.014303               (gift)
 21  0.011919                (lot)
 22  0.020262            (machine)
 23  0.014303              (model)
 24  0.026222              (money)
 25  0.033373              (month)
 26  0.023838              (movie)
 27  0.01311

In [111]:
display_original_pruned_differences(compactness_pruned_features_dict, redundancy_pruned_compactness_features_dict)

,Product,Original Length,Pruned Length,Percentage Reduction (%)
6,Diaper Champ,41,27,34.146341
7,Hitachi router,44,32,27.272727
10,Nikon coolpix 4300,69,53,23.188406
16,norton,41,32,21.951220
2,Canon PowerShot SD500,37,29,21.621622
3,Canon S100,43,37,13.953488
1,Canon G3,67,58,13.432836
14,Speaker,59,52,11.864407
9,MicroMP3,60,54,10.000000
12,Nokia 6610,50,45,10.000000


This then provides us with the final frequent feature sets

In [350]:
frequent_feature_sets = copy.deepcopy(redundancy_pruned_compactness_features_dict)

frequent_feature_sets

{'Apex AD2600 Progressive-scan DVD player':      support             itemsets
 0   0.020262             (ad2600)
 1   0.019070             (amazon)
 2   0.057211               (apex)
 3   0.015495              (brand)
 4   0.025030             (button)
 5   0.010727                (buy)
 6   0.013111                 (cd)
 7   0.023838          (christmas)
 8   0.010727            (control)
 9   0.027414           (customer)
 10  0.021454                (day)
 11  0.026222               (disc)
 12  0.011919               (disk)
 13  0.014303            (display)
 14  0.181168                (dvd)
 15  0.019070         (everything)
 16  0.036949            (feature)
 17  0.011919             (format)
 18  0.017878                (get)
 19  0.014303               (gift)
 21  0.011919                (lot)
 22  0.020262            (machine)
 23  0.014303              (model)
 24  0.026222              (money)
 25  0.033373              (month)
 26  0.023838              (movie)
 27  0.01311

The feature sets could be improved here quite radically if the itemsets with more than one word in them were in an order that was in English. For example, let's observe the feature sets for Apex AD2600 Progressive-scan DVD player:

In [351]:
frequent_feature_sets['Apex AD2600 Progressive-scan DVD player']

,support,itemsets
0,0.020262,(ad2600)
1,0.019070,(amazon)
2,0.057211,(apex)
3,0.015495,(brand)
4,0.025030,(button)
5,0.010727,(buy)
6,0.013111,(cd)
7,0.023838,(christmas)
8,0.010727,(control)
9,0.027414,(customer)


Itemset number 62 is `(player, dvd)`. When inputting this into the sentiment analysis, it will need to be made into a word that is how English is written or spoken because this is what is more likely to appear in reviews. The phrase `player dvd` is not English but `DVD player` obviously is. A further preprocessing step is therefore required to make sure that the phrasing is correct. This will be done below in the `Preparing Dataframe for Input to Sentiment Analysis` section, because we still need to extract Infrequent Features and merge these with Frequent Features to create the Features Dictionary, first, before we prepare the Feature Dictionary for sentiment analysis.

## Opinion Word Extraction

The primary goal of the Opinion Word Extraction stage in the pipeline is to find adjectives that are in proximity to the frequent features (Hu & Liu, n.d). These adjectives will be leveraged to find any nouns that may be Infrequent Features (which is in the step after this one in the pipeline) (Hu & Liu, n.d).

The description of the below algorithm, which is Opinion Word Extraction, is adapted from Hu & Liu (2004). The steps are as follows:
1. loop through each review sentence and check whether there is a feature for the product in the review sentence.
2. if there is then look for what Hu & Liu (2004) call the effective adjective. This is the adjective which is next to the feature and determines the sentiment of the feature.

In [352]:
class OpinionWordExtractor:
    def __init__(self):
        # we have the frequent feature sets above so we can just store this here for each instance
        self.frequent_feature_sets = frequent_feature_sets
        
    def main(self, df_product_reviews):
        product_object = {}
    
        # iterate over each row in the dataframe
        df_product_reviews.apply(
            lambda row: product_object.setdefault(
                row['product'], []
            ).extend(self.extract_opinion_words_in_review(row)),
            axis=1
        )
        
        return {product: list(set(words)) for product, words in product_object.items()}
        
    
    def extract_opinion_words_in_review(self, row):
        feature_words = self.extract_feature_words_in_review(row['review_preprocessed'], row['product'])
        opinion_words = self.extract_potential_opinion_words(row['line'], row['review_preprocessed'], row['product'], feature_words)
        
        return opinion_words

    def extract_feature_words_in_review(self, review_preprocessed, product):
        feature_for_product = self.frequent_feature_sets[product]['itemsets']
        # convert the review_preprocessed into a list of words without the pos tag since it makes it easier
        # to find the review words. We can use the review_preprocessed at a later time should we need to
        # but this is in preparation for finding the feature words more easily
        review_words = [word for word, pos in review_preprocessed]
        found_features = []
        for itemset in feature_for_product:
            if all(word in review_words for word in itemset):
                found_features.append(' '.join(itemset))

        return found_features

    def extract_potential_opinion_words(self, line, review_preprocessed, product, feature_words):
        feature_for_product = self.frequent_feature_sets[product]['itemsets']
        
        opinion_words = []
        
        for feature in feature_for_product:
            feature_indices = [i for i, (word, pos) in enumerate(review_preprocessed) if word in feature]
    
            for feature_index in feature_indices:
                # nearby adjectives are thos that are found adjacent to the feature
                nearby_adjectives = [word for i, (word, pos) in enumerate(review_preprocessed)
                                     if pos.startswith('JJ') and abs(i - feature_index) == 1]
                opinion_words.extend(nearby_adjectives)
    
        return opinion_words

opinion_words_extracted = OpinionWordExtractor().main(df_product_reviews)

opinion_words_extracted

{'Nokia 6610': ['directional',
  'loud',
  'accessible',
  'comaptible',
  'wonderful',
  'great',
  'hard',
  'ready',
  'dial',
  'important',
  'best',
  'decent',
  'downloadable',
  'little',
  'unhappy',
  'build',
  'unlock',
  's105',
  'clear',
  'free',
  'favorite',
  'defective',
  'regular',
  'positive',
  'unchangeable',
  'toy',
  'hiss',
  'first',
  'expensive',
  'automatic',
  'battery',
  'fm',
  'polymorphic',
  'signal',
  'high',
  'revive',
  'nokia',
  'bulky',
  'mobile',
  'beatiful',
  'useful',
  'last',
  'whole',
  'laptop',
  'unlimited',
  'easy',
  'flip',
  'many',
  'optional',
  'slick',
  'propietary',
  'good',
  'brief',
  'current',
  'etc',
  'average',
  'adjust',
  'corporate',
  'tech',
  'european',
  'compose',
  'exceptional',
  'full',
  'multiple',
  'drop',
  'listen',
  'numerous',
  'beautiful',
  'live',
  'awesome',
  'right',
  'previous',
  'nice',
  'visible',
  'stereo',
  'fairness',
  'international',
  'american',
  'suburb

### Only use opinion words that are positive or negative
This is a slight diversion from Hu & Liu (2004), but I found that if too many neutral adjectives were in this stage then the number of Infrequent Features increased significantly and many of them were not features at all. For this reason, the neutral words were removed. This may cause some overfitting, which essentially means that new data that is given to a machine learning model gives inaccurate predictions because the training data was too closely tailored to specific examples it was trained on (Amazon, n.d). However in this context, there is a greater benefit to be had from reducing noise and this outweighs the potential drawback of overfitting. If there was scope to test out the different approaches, it would be beneficial to see whether including the neutral adjectives enhanced or reduced the performance of the classification models.

In [353]:
class OpinionWordPolarity:
    def main(self):
        pass

    def classify_word(self, word):
        testimonial = TextBlob(word)
        polarity = testimonial.sentiment.polarity
        if polarity >= 0.5:
            return 1
        elif polarity <= -0.5:
            return -1
        else:
            return 0

    def get_sentiment_string(self, score):
        if score == 1:
            return 'positive'
        elif score == -1:
            return 'negaitve'

        return 'neutral' 

In [354]:
opinion_words_extracted_with_pos_and_neg_words = {}

opinion_word_polarity = OpinionWordPolarity()
for product_name, word_list in opinion_words_extracted.items():
    pos_and_neg_word_list = []
    
    for word in word_list:
        pos_or_neg_polarity = opinion_word_polarity.classify_word(word)
        if pos_or_neg_polarity != 0:
            pos_and_neg_word_list.append(word)
            
    opinion_words_extracted_with_pos_and_neg_words[product_name] = pos_and_neg_word_list


opinion_words_extracted_with_pos_and_neg_words

{'Nokia 6610': ['wonderful',
  'great',
  'best',
  'unhappy',
  'favorite',
  'expensive',
  'many',
  'good',
  'exceptional',
  'beautiful',
  'awesome',
  'nice',
  'excellent',
  'pleased',
  'bad',
  'sure',
  'attractive',
  'perfect',
  'bright',
  'happy',
  'okay',
  'horrible',
  'superb',
  'able',
  'amazing',
  'cute'],
 'norton': ['disappointment',
  'great',
  'lousy',
  'best',
  'awful',
  'effective',
  'many',
  'sickly',
  'good',
  'useless',
  'terrible',
  'appropriate',
  'excellent',
  'bad',
  'ok',
  'glad',
  'hate',
  'phenomenal',
  'able',
  'disappointed'],
 'Nokia 6600': ['flashy',
  'great',
  'best',
  'unhappy',
  'disappointing',
  'handy',
  'wrong',
  'many',
  'good',
  'useless',
  'beautiful',
  'nice',
  'incredible',
  'excellent',
  'bad',
  'popular',
  'suitable',
  'sure',
  'perfect',
  'bright',
  'ok',
  'able',
  'unfortunate',
  'disappointed'],
 'Hitachi router': ['great',
  'best',
  'top',
  'outstanding',
  'skeptical',
  'good'

## Infrequent Feature Identification

In this stage of the pipeline, infrequent product features are identified in reviews by analysing noun-adjective pairs (Hu & Liu, 2004). It checks if a noun, not already in the list of frequent features, is near sentiment bearing adjectives within a specified proximity (p_value) (Hu & Liu, 2004). If such adjectives are found, the the noun is flagged as an infrequent feature. The process iterates over all reviews, collecting these features for each product and duplicates are removed to ensure unique entries. This stage of the pipeline helps uncover less common but potentially important features associated with customer sentiment (Hu & Liu, 2004).

In [355]:
class InfrequentFeatureIdentifier:
    def __init__(self, frequent_feature_sets, p_value = 1):
        self.frequent_feature_sets = frequent_feature_sets
        self.p_value = p_value

    def main(self, df_product_reviews):
        infrequent_features_dict = {}
        
        for index, row in df_product_reviews.iterrows():
            infrequent_features_dict[row['product']] = self.identify_infrequent_features_for_product(
                row['product'], row['review_preprocessed']
            )

        # if there are any infrequent features that have been found more than once, then we want to remove them
        # since it is no use having the same feature name twice
        for product in infrequent_features_dict.keys():
            infrequent_features_dict[product] = list(set(infrequent_features_dict[product]))
            
        return infrequent_features_dict
         
    def identify_infrequent_features_for_product(self, product, review_preprocessed):
        frequent_features = self.frequent_feature_sets.get(product, {}).get('itemsets', [])
        opinion_words_for_product = opinion_words_extracted_with_pos_and_neg_words[product]
        infrequent_features = []
        
        # here we want to loop through the preprocessed reviews and find any noun and adjective pairs
        for i, (word, pos) in enumerate(review_preprocessed):
            # only look for nouns that are not in the frequent features
            if pos.startswith('NN') and word not in frequent_features:
                nearby_adjectives = [adj_word for adj_word, adj_pos in 
                                     # here we are using the p_value to try to find more or less infrequent features 
                                     review_preprocessed[max(0, i - self.p_value):min(len(review_preprocessed), i + self.p_value)]
                                     if adj_pos.startswith('JJ')]

                nearby_pos_neg_adjectives = []
                for nearby_adjective in nearby_adjectives:
                    for opinion_word_for_product in opinion_words_for_product:
                        if nearby_adjective == opinion_word_for_product:
                            nearby_pos_neg_adjectives.append(opinion_word_for_product)
                            
                # if we found nearby adjectives, consider it an infrequent feature
                if len(nearby_pos_neg_adjectives) > 0:
                    infrequent_features.append(word)
         
        return infrequent_features

Below are a number of different attempts to try to find what the best p_value was:

In [356]:
infrequent_features_one_p_value = InfrequentFeatureIdentifier(frequent_feature_sets).main(copy.deepcopy(df_product_reviews))
infrequent_features_one_p_value

{'Nokia 6610': [],
 'norton': [],
 'Nokia 6600': [],
 'Hitachi router': [],
 'Canon G3': ['camera'],
 'ipod': [],
 'Diaper Champ': [],
 'Linksys Router': [],
 'Creative Labs Nomad Jukebox Zen Xtra 40GB': [],
 'Computer': [],
 'Apex AD2600 Progressive-scan DVD player': [],
 'Speaker': ['speaker'],
 'Nikon coolpix 4300': ['camera', 'travel'],
 'Canon S100': [],
 'MicroMP3': [],
 'Router': [],
 'Canon PowerShot SD500': ['camera']}

In [357]:
infrequent_features_two_p_value = InfrequentFeatureIdentifier(frequent_feature_sets, 2).main(copy.deepcopy(df_product_reviews))

infrequent_features_two_p_value

{'Nokia 6610': [],
 'norton': ['bacon', 'product'],
 'Nokia 6600': [],
 'Hitachi router': [],
 'Canon G3': ['semi', 'camera', 'shoot'],
 'ipod': [],
 'Diaper Champ': [],
 'Linksys Router': [],
 'Creative Labs Nomad Jukebox Zen Xtra 40GB': [],
 'Computer': [],
 'Apex AD2600 Progressive-scan DVD player': [],
 'Speaker': ['speaker'],
 'Nikon coolpix 4300': ['money', 'camera', 'image', 'travel', 'opinion'],
 'Canon S100': [],
 'MicroMP3': [],
 'Router': [],
 'Canon PowerShot SD500': ['camera']}

In [358]:
infrequent_features_three_p_value = InfrequentFeatureIdentifier(frequent_feature_sets, 3).main(copy.deepcopy(df_product_reviews))

infrequent_features_three_p_value

{'Nokia 6610': [],
 'norton': ['bacon', 'product'],
 'Nokia 6600': [],
 'Hitachi router': [],
 'Canon G3': ['pro', 'point', 'semi', 'camera', 'shoot'],
 'ipod': [],
 'Diaper Champ': [],
 'Linksys Router': [],
 'Creative Labs Nomad Jukebox Zen Xtra 40GB': [],
 'Computer': [],
 'Apex AD2600 Progressive-scan DVD player': [],
 'Speaker': ['speaker'],
 'Nikon coolpix 4300': ['use',
  'money',
  'camera',
  'image',
  'travel',
  'opinion'],
 'Canon S100': [],
 'MicroMP3': [],
 'Router': [],
 'Canon PowerShot SD500': ['one', 'camera']}

A p_value of 2 was selected because it effectively balances capturing important contextual information without introducing too much noise. With a p_value of 1, the model might overlook relevant adjectives that are slightly further from the noun, while a p_value of 3 seems to introduce more unrelated words such as 'pro' and 'use', leading to less precise results. Therefore, arguably a p_value 2 provides enough context to identify meaningful infrequent features while minimising the risk of incorporating irrelevant data.

In [359]:
infrequent_features = infrequent_features_two_p_value

## Features Dictionary

In [360]:
features_dict = {}

def frozenset_to_string(fset):
    return ', '.join(fset)
    
for product, df in frequent_feature_sets.items():
    itemsets_array = df['itemsets'].tolist()
    features_dict[product] = itemsets_array
    features_dict[product] = [frozenset_to_string(fset) for fset in features_dict[product]]
    for infrequent_feature in infrequent_features[product]:
        features_dict[product].append(infrequent_feature)

features_dict

{'Apex AD2600 Progressive-scan DVD player': ['ad2600',
  'amazon',
  'apex',
  'brand',
  'button',
  'buy',
  'cd',
  'christmas',
  'control',
  'customer',
  'day',
  'disc',
  'disk',
  'display',
  'dvd',
  'everything',
  'feature',
  'format',
  'get',
  'gift',
  'lot',
  'machine',
  'model',
  'money',
  'month',
  'movie',
  'number',
  'output',
  'picture',
  'play',
  'player',
  'price',
  'problem',
  'product',
  'purchase',
  'quality',
  'remote',
  'return',
  'review',
  'service',
  'something',
  'support',
  'thing',
  'time',
  'tv',
  'unit',
  'use',
  'video',
  'way',
  'week',
  'work',
  'apex, dvd',
  'player, apex',
  'service, customer',
  'play, dvd',
  'player, dvd',
  'play, player',
  'player, apex, dvd'],
 'Canon G3': ['auto',
  'battery',
  'button',
  'buy',
  'camera',
  'canon',
  'card',
  'choice',
  'color',
  'control',
  'day',
  'everything',
  'feel',
  'film',
  'flash',
  'flaw',
  'focus',
  'g2',
  'g3',
  'get',
  'hand',
  'image'

### Preparing Data for Input to Sentiment Analysis

In this section, we need to create a copy of the original dataframe and use the features we have extracted to label each of the review sentences with potential feature we have found in the preceding steps in the pipeline. As mentioned previously, there is a problem with some of the features that have more than one word in them. For example the `MicroMP3` has a feature called `player, mp3`. Obviously this should be `mp3 player`. There are multiple instances of this problem that we need to fix in order to help us classify these review sentences.

One approach I could have taken was to use an LLM to infer the meaning of the phrase (TechLatest.net, 2023). However, installing one that is accurate onto a Jupyter notebook is computationally expensive, and would require a higher amount of GPU's to run, which is why these kinds of tasks are often done in the Cloud (TechLatest.net, 2023). Instead, if a feature has more than one word in it, I have opted to simply check whether those words appear next to one another in the review or title sentence, and if they do, then I would use the ordering of the word or title sentence as the feature to look for. This way it is using natural language to determine if it is a feature or not. There is a possibility that there are some mistakes. But there are multiple instances of the features appearing in the incorrect order.

It is also important to split the data so that it does not include anything that is in the test data (see below for test data). This is because in order to train the model that generalises accurately to new data, we need to make sure that the data that was used to train the model or improve it is not included in any unseen data (which is the testing data) (encord.com, n.d.).

In [361]:
df_testing_data = copy.deepcopy(df_product_reviews)
df_testing_data = df_testing_data[df_testing_data['annotated_labels'] == '']

def extract_noun_and_noun_groups_singular(pos_tagged_review):
    return [word for word, pos in pos_tagged_review if pos.startswith('NN') or pos.startswith('NG')]
    
def check_feature_in_review(row):
    # some compound features are saved such as "life, battery", which should read as battery life.
    # regardless of this, we need to remove the commas and then try to find the phrase in the review
    # If they're next to each other in any order then we can assume they are the correct word
    review_text = " ".join(extract_noun_and_noun_groups_singular(row['review_preprocessed']))
    features_for_product = features_dict[row['product']]
    features_more_than_one_word = get_features_more_than_one_word(features_for_product)
    features_one_word = get_features_one_word(features_for_product)
    
    one_word_aspects = find_one_word_aspects(review_text, features_one_word)
    multiple_word_aspects = find_multiple_word_aspects(review_text, features_more_than_one_word)
    
    # I want to use an array here so that is possible to use the explode method and create more series
    return one_word_aspects + multiple_word_aspects

        
def get_features_more_than_one_word(features_for_product):
    features_with_more_than_one_word = [feature.split(',') for feature in features_for_product if ',' in feature]
    
    return features_with_more_than_one_word

def get_features_one_word(features_for_product):
    features_one_word = [feature for feature in features_for_product if ',' not in feature]
    
    return features_one_word
    
def find_one_word_aspects(review_text, features_one_word):
    features_found = []
    for feature in features_one_word:
        if feature in review_text:
            features_found.append(feature)

    return features_found

def find_multiple_word_aspects(review_text, features_more_than_one_word):
    features_found = []
    for feature in features_more_than_one_word:
        permutations = list(itertools.permutations(feature))
        
        for perm in permutations:
            joined_feature = " ".join(perm)
            if joined_feature in review_text:
                features_found.append(joined_feature.strip())
                # if it's been found then we break because we don't need to find the feature again
                break
    
    return features_found
    
df_testing_data['aspect'] = df_testing_data.apply(lambda row: check_feature_in_review(row), axis=1)

if we inspect the data now, we can see that the `player, dvd` problem that we saw before has now been resolved:

In [362]:
df_testing_data[df_testing_data['review'].str.contains("dvd player")]

,product,line,annotated_labels,review,review_preprocessed,aspect
6496,Apex AD2600 Progressive-scan DVD player,##does your apex dvd player only play dvd audi...,,does your apex dvd player only play dvd audio ...,"[(apex, NN), (dvd, NN), (player, NN), (play, N...","[apex, dvd, play, player, video, dvd play, dvd..."
6509,Apex AD2600 Progressive-scan DVD player,##the p/n button switches your dvd players vid...,,the p/n button switches your dvd players video...,"[(pn, JJ), (button, NN), (switch, NNS), (dvd, ...","[button, output, play, player]"
6512,Apex AD2600 Progressive-scan DVD player,##i hope i have helped you continue to enjoy t...,,i hope i have helped you continue to enjoy the...,"[(hope, NN), (help, VBD), (continue, VB), (enj...","[dvd, play, player, dvd play, dvd player]"
6522,Apex AD2600 Progressive-scan DVD player,##i 've owned 6 or 7 dvd players since 1998 .,,i 've owned 6 or 7 dvd players since 1998 .,"[(own, VBD), (6, CD), (7, CD), (dvd, NN), (pla...","[dvd, play, player]"
6577,Apex AD2600 Progressive-scan DVD player,##i 've bought 3 apex dvd players now .,,i 've bought 3 apex dvd players now .,"[(buy, VBD), (3, CD), (apex, JJ), (dvd, NN), (...","[dvd, play, player]"
6610,Apex AD2600 Progressive-scan DVD player,##while it seems like some people have had tro...,,while it seems like some people have had troub...,"[(seem, VBZ), (like, IN), (people, NNS), (trou...","[play, player, purchase]"
6647,Apex AD2600 Progressive-scan DVD player,[t] ok if you have a 2nd dvd player .,,ok if you have a 2nd dvd player .,"[(ok, JJ), (2nd, CD), (dvd, NN), (player, NN)]","[dvd, play, player]"
6648,Apex AD2600 Progressive-scan DVD player,##i have had this dvd player for about 3 month...,,i have had this dvd player for about 3 months ...,"[(dvd, NN), (player, NN), (3, CD), (month, NNS)]","[dvd, month, play, player]"
6708,Apex AD2600 Progressive-scan DVD player,##features and looks this dvd player is overpr...,,features and looks this dvd player is overpric...,"[(feature, NNS), (look, VBZ), (dvd, JJ), (play...","[brand, feature, play, player]"
6727,Apex AD2600 Progressive-scan DVD player,[t] avoid apex dvd players - even if they 're ...,,avoid apex dvd players - even if they 're free !,"[(avoid, JJ), (apex, NN), (dvd, NN), (player, ...","[apex, dvd, play, player, dvd play, dvd player]"


In [363]:
df_testing_data = df_testing_data.explode('aspect')
df_testing_data

,product,line,annotated_labels,review,review_preprocessed,aspect
0,Nokia 6610,"[t]excellent phone , excellent service .",,"excellent phone , excellent service .","[(excellent, JJ), (phone, NN), (excellent, JJ)...",phone
0,Nokia 6610,"[t]excellent phone , excellent service .",,"excellent phone , excellent service .","[(excellent, JJ), (phone, NN), (excellent, JJ)...",service
1,Nokia 6610,##i am a business user who heavily depend on m...,,i am a business user who heavily depend on mob...,"[(business, NN), (user, VBD), (heavily, RB), (...",service
3,Nokia 6610,##just double check with customer service to e...,,just double check with customer service to ens...,"[(double, RB), (check, VB), (customer, NN), (s...",customer
3,Nokia 6610,##just double check with customer service to e...,,just double check with customer service to ens...,"[(double, RB), (check, VB), (customer, NN), (s...",number
...,...,...,...,...,...,...
10911,Canon PowerShot SD500,##On the 60fps setting it seems like a regular...,,On the 60fps setting it seems like a regular v...,"[(60fps, CD), (set, VBG), (seem, VBZ), (like, ...",video
10913,Canon PowerShot SD500,##An A++!!!,,An A++!!!,[],NaN
10916,Canon PowerShot SD500,##It seems you have to double click some of th...,,It seems you have to double click some of the ...,"[(seem, VBZ), (double, JJ), (click, JJ), (time...",time
10917,Canon PowerShot SD500,##Huh? I missed out on lots of photos and lots...,,Huh? I missed out on lots of photos and lots a...,"[(huh, NNS), (miss, VBD), (lot, NNS), (photo, ...",lot


In [422]:
# this removes NaN and None from the aspect column
df_testing_data['aspect'] = df_testing_data['aspect'].fillna('')
df_testing_data

,product,line,annotated_labels,review,review_preprocessed,aspect
0,Nokia 6610,"[t]excellent phone , excellent service .",,"excellent phone , excellent service .","[(excellent, JJ), (phone, NN), (excellent, JJ)...",phone
0,Nokia 6610,"[t]excellent phone , excellent service .",,"excellent phone , excellent service .","[(excellent, JJ), (phone, NN), (excellent, JJ)...",service
1,Nokia 6610,##i am a business user who heavily depend on m...,,i am a business user who heavily depend on mob...,"[(business, NN), (user, VBD), (heavily, RB), (...",service
3,Nokia 6610,##just double check with customer service to e...,,just double check with customer service to ens...,"[(double, RB), (check, VB), (customer, NN), (s...",customer
3,Nokia 6610,##just double check with customer service to e...,,just double check with customer service to ens...,"[(double, RB), (check, VB), (customer, NN), (s...",number
...,...,...,...,...,...,...
10911,Canon PowerShot SD500,##On the 60fps setting it seems like a regular...,,On the 60fps setting it seems like a regular v...,"[(60fps, CD), (set, VBG), (seem, VBZ), (like, ...",video
10913,Canon PowerShot SD500,##An A++!!!,,An A++!!!,[],
10916,Canon PowerShot SD500,##It seems you have to double click some of th...,,It seems you have to double click some of the ...,"[(seem, VBZ), (double, JJ), (click, JJ), (time...",time
10917,Canon PowerShot SD500,##Huh? I missed out on lots of photos and lots...,,Huh? I missed out on lots of photos and lots a...,"[(huh, NNS), (miss, VBD), (lot, NNS), (photo, ...",lot


# 2. Sentiment Analysis

This report is going to use Aspect Based Sentiment Analysis (ABSA) to try to determine the either positive or negative sentiment of specific features (Chusiano, 2022). Typical sentiment analysis involves analysing an entire sentence and determining whether the sentence is either positive or negative (Chusiano, 2022). This is useful most the time but there are also many times where sentences can have opposite polarities for different features so classifying all the features in a sentence with one polarity would be inaccurate (Chusiano, 2022). This is where ABSA is useful since it can determine the polarity of individual features within sentences (Chusiano, 2022). The previous feature extraction steps are crucial for ABSA because they identify the specific aspects or features within the text, allowing the model to determine the sentiment associated with each feature individually.

There will be a comparitive assessment between two different sentiment analysis classification models in this section. The first will be a rules based ABSA approach (Aris Pattakos, 2021) and the second will use a pre-trained model that will use a supervised learning approach. The justification for why these are very different approaches and why a supervised learning approach will be used on top of the rules based approach will become apparent as you read through the report.

#### Product Summary helper

This is a helper class to print out the product summaries for the different models

In [336]:
class ProductSummary:
    def __init__(self):
        self.summary = self.create
        
    def create(self):
        for product_name, group in df_product_reviews.groupby('product'):
            if product_name not in self.product_summary:
                 self.product_summary[product_name] = {}
          
            for feature in features_dict[product_name]:
                 self.product_summary[product_name][feature] = {'positive': 0, 'negative': 0, 'examples': {'positive': [], 'negative': []}}

        return product_summary

    def add_review(self, review, polarity):
         if sentiment in self.summary[product_name][feature]:
            self.summary[product_name][feature][polarity] += 1
            self.summary[product_name][feature]["examples"][polarity].append(review)


### Groundtruth labels

We have some groundtruth labels in the dataset which were extracted in Parse Reviews and stored under `annotated_labels` in the Review Database. We will use these in order to test the differences between the classification models. This means we can better analyse the performance of the classification models by comparing how many features were correctly labeled when using the same training data (Understand.ai, n.d). Below we are adding a new column called `encoded_label` to the Reviews Database. Where there are multiple annotated labels, these will be separated into however many annotated labels there are to reflect that each review sentence can have multiple features. 

In [282]:
df_training_data = df_product_reviews[df_product_reviews['annotated_labels'] !='']
df_training_data['label'] = df_training_data['annotated_labels'].apply(lambda x: x.split(','))
df_training_data = df_training_data.explode('label')
df_training_data['aspect'] = df_training_data['label'].apply(lambda x: re.sub(r"\[\+?\-?\d+\]", "", x))
df_training_data['encoded_aspect'] = df_training_data['annotated_labels'].apply(lambda x: 1 if '+' in x else -1)
df_training_data.head()

/var/folders/1z/gqg96lxd46q_83681gtwy4980000gp/T/ipykernel_58248/332186527.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_training_data['label'] = df_training_data['annotated_labels'].apply(lambda x: x.split(','))


,product,line,annotated_labels,review,review_preprocessed,label,aspect,encoded_aspect
2,Nokia 6610,"phone[+3], work[+2]##there is much which has b...","phone[+3], work[+2]",there is much which has been said in other rev...,"[(much, RB), (say, VBD), (reviews, JJ), (featu...",phone[+3],phone,1
2,Nokia 6610,"phone[+3], work[+2]##there is much which has b...","phone[+3], work[+2]",there is much which has been said in other rev...,"[(much, RB), (say, VBD), (reviews, JJ), (featu...",work[+2],work,1
4,Nokia 6610,at&t customer service[-2]##after several years...,at&t customer service[-2],after several years of torture in the hands of...,"[(several, JJ), (year, NNS), (torture, NN), (h...",at&t customer service[-2],at&t customer service,-1
5,Nokia 6610,signal quality[+3]##i have had the phone for 1...,signal quality[+3],"i have had the phone for 1 week , the signal q...","[(phone, NN), (1, CD), (week, NN), (signal, JJ...",signal quality[+3],signal quality,1
7,Nokia 6610,"speaker phone[+2],radio[+2],infrared[+2]##my f...","speaker phone[+2],radio[+2],infrared[+2]","my favorite features , although there are many...","[(favorite, JJ), (feature, NNS), (although, IN...",speaker phone[+2],speaker phone,1


#### There are class imbalances in the groundtruth labels
It is worth highlighting that there are class imbalances in the groundtruth labels. Class imbalances describes a problem where the number of entities classified in one class is significantly skewed in its favour compared to other classes (Pilot, 2023). In this instance, there are far more positive labels in the groundtruth labels than negative ones. This can lead to there being more classifications towards the positive class than the negative class (Pilot, 2023).

In [283]:
df_training_data.encoded_aspect.value_counts()

encoded_aspect
 1    4030
-1    1818
Name: count, dtype: int64

### Rules Based Aspect Based Sentiment Analysis
The first sentiment analysis approach uses a very simple rules based approach algorithm. A rules based approach often uses PoS tags to determine adjectives that are close to aspects (or features) in text (Aris Pattakos, 2021). This particular algorithm is based on the most basic algorithm used used in Aris Pattakos (2021) and works in the following way:
1. pass in a row from a dataframe which has preprocessed PoS tags in it and the potential aspects in there
2. if the current word is an adjective, then check the next word and determine if it is the first word of the aspect (there can be more than one word in an aspect)
3. if it is, then determine the sentiment of the adjective and add this to a column called `encoded_aspect`
4. if it is not then add None to the `encoded_aspect` column. This denotes that there was no descriptor found for the aspect

In [365]:
def classify_sentiment(polarity):
    if polarity > 0:
        return 1
    elif polarity < 0:
        return -1

    return 0
        
def rules_based_algorithm(row):
    review_text = row['review_preprocessed']
    for i, word_pos_tag_tuple in enumerate(review_text):
        word, pos_tag = word_pos_tag_tuple
        if i + 1 >= len(review_text) or not row['aspect']:
            continue

        # if the pos_tag is an adjective and the word next to the adjective is the
        # aspect word, then get its sentiment score
        next_word = review_text[i + 1][0]
        # some aspects have multiple words in them so we want to only get the first
        # word and see if the word next to the adjective is the word itself
        first_word_of_aspect = row['aspect'].split(' ')[0]
        if pos_tag == 'JJ' and next_word == first_word_of_aspect:
            # calculate sentiment for the adjective
            sentiment = TextBlob(word).sentiment
            
            return classify_sentiment(sentiment.polarity)

    return None

In [366]:
df_base_classifier_test = copy.deepcopy(df_testing_data)
df_base_classifier_test['encoded_aspect'] = df_testing_data.apply(lambda row: rules_based_algorithm(row), axis=1)

In [306]:
df_base_classifier_test

,product,line,annotated_labels,review,review_preprocessed,aspect,encoded_aspect
0,Nokia 6610,"[t]excellent phone , excellent service .",,"excellent phone , excellent service .","[(excellent, JJ), (phone, NN), (excellent, JJ)...",service,1.0
1,Nokia 6610,##i am a business user who heavily depend on m...,,i am a business user who heavily depend on mob...,"[(business, NN), (user, VBD), (heavily, RB), (...",service,0.0
2,Nokia 6610,"phone[+3], work[+2]##there is much which has b...","phone[+3], work[+2]",there is much which has been said in other rev...,"[(much, RB), (say, VBD), (reviews, JJ), (featu...",feature,0.0
2,Nokia 6610,"phone[+3], work[+2]##there is much which has b...","phone[+3], work[+2]",there is much which has been said in other rev...,"[(much, RB), (say, VBD), (reviews, JJ), (featu...",problem,NaN
3,Nokia 6610,##just double check with customer service to e...,,just double check with customer service to ens...,"[(double, RB), (check, VB), (customer, NN), (s...",customer,NaN
...,...,...,...,...,...,...,...
10915,Canon PowerShot SD500,automatic shutter[-3]##There were many times w...,automatic shutter[-3],There were many times when I clicked the autom...,"[(many, JJ), (time, NNS), (click, VBD), (autom...",,NaN
10916,Canon PowerShot SD500,##It seems you have to double click some of th...,,It seems you have to double click some of the ...,"[(seem, VBZ), (double, JJ), (click, JJ), (time...",,NaN
10917,Canon PowerShot SD500,##Huh? I missed out on lots of photos and lots...,,Huh? I missed out on lots of photos and lots a...,"[(huh, NNS), (miss, VBD), (lot, NNS), (photo, ...",lot,NaN
10918,Canon PowerShot SD500,controls[-1]##The controls are hard to manipul...,controls[-1],"The controls are hard to manipulate, especiall...","[(control, NNS), (hard, JJ), (manipulate, NN),...",,NaN


In [367]:
df_base_classifier_train = copy.deepcopy(df_training_data)
df_base_classifier_train['encoded_aspect'] = df_training_data.apply(lambda row: rules_based_algorithm(row), axis=1)

In [368]:
df_base_classifier_train

,product,line,annotated_labels,review,review_preprocessed,label,aspect,encoded_aspect
2,Nokia 6610,"phone[+3], work[+2]##there is much which has b...","phone[+3], work[+2]",there is much which has been said in other rev...,"[(much, RB), (say, VBD), (reviews, JJ), (featu...",phone[+3],phone,1.0
2,Nokia 6610,"phone[+3], work[+2]##there is much which has b...","phone[+3], work[+2]",there is much which has been said in other rev...,"[(much, RB), (say, VBD), (reviews, JJ), (featu...",work[+2],work,NaN
4,Nokia 6610,at&t customer service[-2]##after several years...,at&t customer service[-2],after several years of torture in the hands of...,"[(several, JJ), (year, NNS), (torture, NN), (h...",at&t customer service[-2],at&t customer service,NaN
5,Nokia 6610,signal quality[+3]##i have had the phone for 1...,signal quality[+3],"i have had the phone for 1 week , the signal q...","[(phone, NN), (1, CD), (week, NN), (signal, JJ...",signal quality[+3],signal quality,1.0
7,Nokia 6610,"speaker phone[+2],radio[+2],infrared[+2]##my f...","speaker phone[+2],radio[+2],infrared[+2]","my favorite features , although there are many...","[(favorite, JJ), (feature, NNS), (although, IN...",speaker phone[+2],speaker phone,1.0
...,...,...,...,...,...,...,...,...
10912,Canon PowerShot SD500,fits[+3]##And it fits right in my pocket much ...,fits[+3],And it fits right in my pocket much easier tha...,"[(fit, NNS), (right, RB), (pocket, RB), (much,...",fits[+3],fits,NaN
10914,Canon PowerShot SD500,camera[-1]##I thought this camera was awful an...,camera[-1],I thought this camera was awful and I did tons...,"[(think, VBN), (camera, NN), (awful, JJ), (ton...",camera[-1],camera,NaN
10915,Canon PowerShot SD500,automatic shutter[-3]##There were many times w...,automatic shutter[-3],There were many times when I clicked the autom...,"[(many, JJ), (time, NNS), (click, VBD), (autom...",automatic shutter[-3],automatic shutter,NaN
10918,Canon PowerShot SD500,controls[-1]##The controls are hard to manipul...,controls[-1],"The controls are hard to manipulate, especiall...","[(control, NNS), (hard, JJ), (manipulate, NN),...",controls[-1],controls,NaN


#### Areas for improvement
An obvious problem with the model in this current form in that it does not take into consideration things such as negation words (Hu & Liu, 2004). There is an opportunity here to introduce looking at the negation of words and whether they adjust the meaning of an adjective that is closest to a feature (Hu & Liu, 2004). For example, if the word 'not' appears before 'good' and that phrase (not good) is describing a feature, then that means the review sentence should be classified as being negative towards that feature (Hu & Liu, 2004). This is actually a problem with aspect based algorithms in general - they fail to spot the nuances and complexities of language and often are far less accurate than other methods (Sayedi, 2021). This is where a supervised learning approach will be better suited since these models are trained on a large corpus of data and can understand nuances better (IBM, 2023).

### Supervised Learning Aspect-Based Sentiment Analysis
Now I will compare the performance of the previous classification method with a supervised learning approach. A supervised learning approach uses a pretrained model to determine the sentiment of given feature (IBM, 2023). There are many pretrained models that can be used to perform sentiment analysis from `hugging face` (Chusiano, 2022). The model used here is `yangheng/deberta-v3-base-absa-v1.1` which can be found here at huggingface.co (n.d).

A supervised learning approach is trained to perform specific tasks based on the training data it is provided (IBM, 2023). In the case of the model used in this report (`yangheng/deberta-v3-base-absa-v1.1`), this model was trained on over 180k examples of labelled data that gave polarity to particular aspects from sentiments (huggingface.co, n.d). 

A supervised learning approach uses labeled data to train a model and thereby make predictions (IBM, 2023). The `yangheng/deberta-v3-base-absa-v1.1` model has been pre trained on a large corpus of ABSA datasets (huggingface.co, n.d). The algorithm below is simple, in that for each review sentence, the features from the feature_dict are found within the review sentences and provided to the `yangheng/deberta-v3-base-absa-v1.1` model along with the review sentence. For each of the aspects provided from the feature_dict, the model outputs the predicted sentiment for that particular aspect. This approach is significantly different to the previous rule based method because it uses deep learning and huge corpus of data to infer complex relationships in the data, which should lead to more accurate sentiment analysis (IBM, 2023).

In [274]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import multiprocessing as mp

model_name = "yangheng/deberta-v3-base-absa-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

/Users/tomhill/anaconda3/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [293]:
def classify_sentiment_supervised_learning(polarity_label):
    if polarity_label == 'Positive':
        return 1
    elif polarity_label == 'Negative':
        return -1

    return 0

def supervised_learning_approach(row):
    review_text = row['review']
    aspect = row['aspect']
    if aspect:
        polarity = classifier(review_text,  text_pair=aspect)
        
        return classify_sentiment_supervised_learning(polarity[0]['label'])
        
    return None

In [292]:
df_supervised_learning_classifier_test = copy.deepcopy(df_testing_data)
df_supervised_learning_classifier_test['encoded_aspect'] = df_testing_data.apply(lambda row: supervised_learning_approach(row), axis=1) 

In [295]:
df_supervised_learning_classifier_test

,product,line,annotated_labels,review,review_preprocessed,aspect,encoded_aspect
0,Nokia 6610,"[t]excellent phone , excellent service .",,"excellent phone , excellent service .","[(excellent, JJ), (phone, NN), (excellent, JJ)...",service,1.0
1,Nokia 6610,##i am a business user who heavily depend on m...,,i am a business user who heavily depend on mob...,"[(business, NN), (user, VBD), (heavily, RB), (...",service,1.0
2,Nokia 6610,"phone[+3], work[+2]##there is much which has b...","phone[+3], work[+2]",there is much which has been said in other rev...,"[(much, RB), (say, VBD), (reviews, JJ), (featu...",feature,0.0
2,Nokia 6610,"phone[+3], work[+2]##there is much which has b...","phone[+3], work[+2]",there is much which has been said in other rev...,"[(much, RB), (say, VBD), (reviews, JJ), (featu...",problem,1.0
3,Nokia 6610,##just double check with customer service to e...,,just double check with customer service to ens...,"[(double, RB), (check, VB), (customer, NN), (s...",customer,0.0
...,...,...,...,...,...,...,...
10915,Canon PowerShot SD500,automatic shutter[-3]##There were many times w...,automatic shutter[-3],There were many times when I clicked the autom...,"[(many, JJ), (time, NNS), (click, VBD), (autom...",,NaN
10916,Canon PowerShot SD500,##It seems you have to double click some of th...,,It seems you have to double click some of the ...,"[(seem, VBZ), (double, JJ), (click, JJ), (time...",,NaN
10917,Canon PowerShot SD500,##Huh? I missed out on lots of photos and lots...,,Huh? I missed out on lots of photos and lots a...,"[(huh, NNS), (miss, VBD), (lot, NNS), (photo, ...",lot,-1.0
10918,Canon PowerShot SD500,controls[-1]##The controls are hard to manipul...,controls[-1],"The controls are hard to manipulate, especiall...","[(control, NNS), (hard, JJ), (manipulate, NN),...",,NaN


In [298]:
df_supervised_learning_classifier_train = copy.deepcopy(df_training_data)
df_supervised_learning_classifier_train['encoded_aspect'] = df_training_data.apply(lambda row: supervised_learning_approach(row), axis=1)

In [369]:
df_supervised_learning_classifier_train

,product,line,annotated_labels,review,review_preprocessed,label,aspect,encoded_aspect
2,Nokia 6610,"phone[+3], work[+2]##there is much which has b...","phone[+3], work[+2]",there is much which has been said in other rev...,"[(much, RB), (say, VBD), (reviews, JJ), (featu...",phone[+3],phone,1
2,Nokia 6610,"phone[+3], work[+2]##there is much which has b...","phone[+3], work[+2]",there is much which has been said in other rev...,"[(much, RB), (say, VBD), (reviews, JJ), (featu...",work[+2],work,1
4,Nokia 6610,at&t customer service[-2]##after several years...,at&t customer service[-2],after several years of torture in the hands of...,"[(several, JJ), (year, NNS), (torture, NN), (h...",at&t customer service[-2],at&t customer service,-1
5,Nokia 6610,signal quality[+3]##i have had the phone for 1...,signal quality[+3],"i have had the phone for 1 week , the signal q...","[(phone, NN), (1, CD), (week, NN), (signal, JJ...",signal quality[+3],signal quality,1
7,Nokia 6610,"speaker phone[+2],radio[+2],infrared[+2]##my f...","speaker phone[+2],radio[+2],infrared[+2]","my favorite features , although there are many...","[(favorite, JJ), (feature, NNS), (although, IN...",speaker phone[+2],speaker phone,1
...,...,...,...,...,...,...,...,...
10912,Canon PowerShot SD500,fits[+3]##And it fits right in my pocket much ...,fits[+3],And it fits right in my pocket much easier tha...,"[(fit, NNS), (right, RB), (pocket, RB), (much,...",fits[+3],fits,1
10914,Canon PowerShot SD500,camera[-1]##I thought this camera was awful an...,camera[-1],I thought this camera was awful and I did tons...,"[(think, VBN), (camera, NN), (awful, JJ), (ton...",camera[-1],camera,-1
10915,Canon PowerShot SD500,automatic shutter[-3]##There were many times w...,automatic shutter[-3],There were many times when I clicked the autom...,"[(many, JJ), (time, NNS), (click, VBD), (autom...",automatic shutter[-3],automatic shutter,-1
10918,Canon PowerShot SD500,controls[-1]##The controls are hard to manipul...,controls[-1],"The controls are hard to manipulate, especiall...","[(control, NNS), (hard, JJ), (manipulate, NN),...",controls[-1],controls,-1


### Comparative Analysis

In [324]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

The comparative analysis will be performed only on the training data, since we can compare more accurately how the model performed given each model is given the same features and we have a groundtruth to compare it against. Before performing any comparative analysis, it is important that we remove any of the `None` from the labels since `sklearn`, which we will use to compare how well the models performed numerically, needs the columns to be in nu In the examples above, we fed in the same aspects to each review sentence and each classification model classed each aspect as either positive, negative, neutral, or if it didn't find anything then it would set it to `None`.

In order to find the score itself, we need to remove `None` since sklearn needs numbers to calculate a final accuracy score.

In [329]:
df_supervised_learning_classifier_train['encoded_aspect'].fillna(0, inplace=True)
df_base_classifier_train['encoded_aspect'].fillna(0, inplace=True)
df_training_data['encoded_aspect'].fillna(0, inplace=True)

#### Comparing the training datasets
Obviously the initial training dataset has all of the correctly encoded labels. Anywhere the aspects have not been correctly labelled in either the supervised or rules based approach can be compared against this.

We can start by looking at the accuracy score, which can be done using the `sklearn` library, because it demonstrates, <i>overall</i> how well an approach has scored relative to the groundtruth labels (Educative: Interactive Courses for Software Developers n.d.). 'overall' is emphasised because accuracy only reflects the total number of correct predictions and does not account for whether the correct labels were distributed appropriately across the different classes.

In [330]:
supervised_accuracy = accuracy_score(df_training_data['encoded_aspect'], 
                                     df_supervised_learning_classifier_train['encoded_aspect'])
rules_based_accuracy = accuracy_score(df_training_data['encoded_aspect'], 
                                      df_base_classifier_train['encoded_aspect'])

print(f"supervised learning accuracy: {supervised_accuracy:.2f}")
print(f"rules based accuracy: {rules_based_accuracy:.2f}")

supervised learning accuracy: 0.84
rules based accuracy: 0.08


There is a very large difference between the two accuracy scores. This suggest that the rules based approach is much too simplistic compared with the supervised learning approach. There is, however, room to improve the accuracy scores in the supervised learning approach, which can perhaps be achieved by training the model further.

To delve deeper into some metrics, it is important to also look at the precision, recall and f1-score. The f1-score is made up of precision, which is the ability of the classifier to correctly label a review and in particular, minimising instances where a positive review is labelled as negative (scikit-learn.org, n.d), and recall, which is the ability of the classifier to correctly label all the actual positive reviews as positive (scikit-learn.org2, n.d). These together can give a clearer picture of how well the classifiers performed.

In [327]:
valid_indices = df_training_data['encoded_aspect'].notna()
ground_truth = df_training_data.loc[valid_indices, 'encoded_aspect']

supervised_pred = df_supervised_learning_classifier_train.loc[valid_indices, 'encoded_aspect']
rules_based_pred = df_base_classifier_train.loc[valid_indices, 'encoded_aspect']

def get_f1(dataframe):
    return f1_score(ground_truth, dataframe, average='macro', zero_division=0)
    
def get_recall(dataframe):
    return recall_score(ground_truth, dataframe, average='macro', zero_division=0)
    
def get_precision(dataframe):
    return precision_score(ground_truth, dataframe, average='macro', zero_division=0)
    
print(f"supervised learning - precision: {get_precision(supervised_pred):.2f}, recall: {get_recall(supervised_pred):.2f}, f1-score: {get_f1(supervised_pred):.2f}")

print(f"rules based - precision: {get_precision(rules_based_pred):.2f}, recall: {get_recall(rules_based_pred):.2f}, f1-score: {get_f1(rules_based_pred):.2f}")

supervised learning - precision: 0.61, recall: 0.56, f1-score: 0.58
rules based - precision: 0.45, recall: 0.04, f1-score: 0.08


The supervised learning approach has a higher precision than the rules based approach, meaning it is less likely to make mistakes and label a positive review as negative. Similarly, the supervised learning approach has a much higher recall ability, which means it labels a much larger proprtion of positive reviews as positive compared with the rules based approach. Overall it then has a higher f1-score and therefore is a much better model for this sentiment analysis task.

We can do a quick check to compare the testing sets to see whether there are any significant differences between the two of them

In [411]:
class ProductSummary:
    def __init__(self):
        self.summary = {}
        
        for product_name, group in df_product_reviews.groupby('product'):
            if product_name not in self.summary:
                 self.summary[product_name] = {}
          
            for feature in features_dict[product_name]:
                 self.summary[product_name][feature] = {'positive': 0, 'negative': 0, 'examples': {'positive': [], 'negative': []}}
        
    def add_review(self, row):
        product_name = row['product']
        review = row['review']
        polarity = row['encoded_aspect']
        feature = row['aspect'].strip()
        polarity_string = self.get_string_polarity(polarity)

        if feature not in self.summary[product_name]:
            self.summary[product_name][feature] = {'positive': 0, 'negative': 0, 'examples': {'positive': [], 'negative': []}}
            
        if polarity_string in self.summary[product_name][feature]:
            self.summary[product_name][feature][polarity_string] += 1
            self.summary[product_name][feature]["examples"][polarity_string].append(review)
            
    def get_string_polarity(self, polarity):
        if polarity == 1:
            return 'positive'
        elif polarity == -1:
            return 'negative'

        return 'neutral'

In [418]:
supervised_learning_test_product_summary = ProductSummary()

df_supervised_learning_classifier_test.apply(lambda row: supervised_learning_test_product_summary.add_review(row), axis=1)

supervised_learning_test_product_summary.summary

{'Apex AD2600 Progressive-scan DVD player': {'ad2600': {'positive': 6,
   'negative': 3,
   'examples': {'positive': ["the cool thing about the ad-2600 it plays alot of different file like mp3 , wma cds , jpeg and kodak picture cds , dvd 's , dvd - / + r 's .",
     'i find the apex ad-2600 to be a great buy .',
     'after having bought and been disappointed in another brand of dvd player , i purchased the apex ad2600 from amazon and first of all i should say it was delivered much more quickly than i had expected .',
     'i am very pleased with the apex ad2600.it plays just about anything you put in the drawer .',
     'i received the apex ad-2600 before i expected .',
     ' apex ad-2600 compatability .'],
    'negative': [' apex ad2600 known capadabilty problem .',
     "i have sent my ad2600 back to apex because it wo n't play certain disney and other dvd movies ( no disc message ) .",
     ' the ad2600 had false silver plate instead of play display .']}},
  'amazon': {'positive':

In [421]:
df_base_classifier_test_summary = ProductSummary()

df_base_classifier_test.apply(lambda row: df_base_classifier_test_summary.add_review(row), axis=1)

df_base_classifier_test_summary.summary

{'Apex AD2600 Progressive-scan DVD player': {'ad2600': {'positive': 1,
   'negative': 0,
   'examples': {'positive': ['they sent me a new ad2600 with the same problem .'],
    'negative': []}},
  'amazon': {'positive': 1,
   'negative': 0,
   'examples': {'positive': ['i am surprised amazon still carries this after all of the continued problems .'],
    'negative': []}},
  'apex': {'positive': 0,
   'negative': 1,
   'examples': {'positive': [],
    'negative': ['i have done some research and experimenting with the remote when my apex ad-2500 seemed to have lost its video signal .']}},
  'brand': {'positive': 0,
   'negative': 0,
   'examples': {'positive': [], 'negative': []}},
  'button': {'positive': 0,
   'negative': 1,
   'examples': {'positive': [],
    'negative': ['i think most of you people complaining about no video is due to you hitting the wrong button either the p/n = pal / ntsc or the i/p is set to progressive scan .']}},
  'buy': {'positive': 1,
   'negative': 0,
   'exa

comparing the product `Apex AD2600 Progressive-scan DVD player` (which is the first product), it is possible to cursorily glance and see that the `df_base_classifier_train` also seemingly has a very low accuracy score since it misses many of the classifications that the `df_supervised_learning_classifier_train` model got correct such as "`apex ad2600 known capadabilty problem .`" being classified as a negative review for `ad2600`. This again, showcases how the supervised learning method is a better model at classifying these reviews correctly on unseen data.


# 3. Evaluation & Discussion

It is possible to draw some meaningful conclusions. We can tell from the testing data that the supervised learning method is far superior at classifying reviews from this project. We can also observe from the product summaries that the supervised learning method is, again, far superior at classifying reviews from this project - but this time with unseen data.

There are, however, certainly more opportunities for future research. The groundtruth labels provided by Hu & Liu (n.d) were too different to the features discovered during the feature extraction phase. This discrepancy made it difficult to evaluate the accuracy of the model on unseen data comprehensively. This would require removing the current groundtruth labels and adding ones that are more aligned with the features found in the preprocessing steps. This would also mean that we could use the training datasets to train the supervised learning method model, and then use the testing data to see how it performs on unseen data more accurately.

Throughout the entire system, there were many times where it could have been possible to fine tune certain parameters and test whether these increase or decrease the overall accuracy of the model. There were also opportunities to introduce tables, such as a confusion matrix, and graphs to better explain some of the results. However, the metrics above should suffice in demonstrating that the supervised learning approach is far superior to the very basic rules based ABSA model. There of course is also scope to introduce even more supervised learning methods, such as support vector machines, random forest or other deep learning neural networks that could prove to perform better than the current supervised learning method.

# References
Analytics Vidhya. (2021). Rule-Based Sentiment Analysis in Python for Data Scientists. [online] Available at: https://www.analyticsvidhya.com/blog/2021/06/rule-based-sentiment-analysis-in-python/.

Aris Pattakos (2021). Aspect-Based Sentiment Analysis Using Spacy & TextBlob. [online] Medium. Available at: https://medium.com/p/4c8de3e0d2b9 [Accessed 1 Sep. 2024].

AWS (n.d.). What is Overfitting? - Overfitting - AWS. [online] Amazon Web Services, Inc. Available at: https://aws.amazon.com/what-is/overfitting/.

Bhalla, D. (n.d.). Pointwise mutual information (PMI) in NLP. [online] ListenData. Available at: https://www.listendata.com/2022/06/pointwise-mutual-information-pmi.html.

Chiusano, F. (2022). Quick intro to Aspect-Based Sentiment Analysis. [online] NLPlanet. Available at: https://medium.com/nlplanet/quick-intro-to-aspect-based-sentiment-analysis-c8888a09eda7.

D, A. (2018). Association Rule. [online] GeeksforGeeks. Available at: https://www.geeksforgeeks.org/association-rule/.

Educative: Interactive Courses for Software Developers. (n.d.). What is the accuracy_score function in Sklearn? [online] Available at: https://www.educative.io/answers/what-is-the-accuracyscore-function-in-sklearn.

encord.com. (n.d.). Training, Validation, Test Split for Machine Learning Datasets. [online] Available at: https://encord.com/blog/train-val-test-split.

GeeksforGeeks (2018). Apriori Algorithm - GeeksforGeeks. [online] GeeksforGeeks. Available at: https://www.geeksforgeeks.org/apriori-algorithm/.

Hu, M. and Liu, B. (n.d.). Mining Opinion Features in Customer Reviews. [online] Available at: https://www.cs.uic.edu/~liub/publications/aaai04-featureExtract.pdf.

Hu, M. and Liu, B. (2004). Mining and summarizing customer reviews. Proceedings of the 2004 ACM SIGKDD international conference on Knowledge discovery and data mining - KDD ’04, [online] pp.168–177. doi:https://doi.org/10.1145/1014052.1014073.
‌
huggingface.co. (n.d.). yangheng/deberta-v3-base-absa-v1.1 · Hugging Face. [online] Available at: https://huggingface.co/yangheng/deberta-v3-base-absa-v1.1.

IBM (2023). What is supervised learning? [online] IBM. Available at: https://www.ibm.com/topics/supervised-learning.

Medium. (2024). Medium. [online] Available at: https://towardsdatascience.com/understanding-apriori-algorithm-in-python-from-scratch-3e97af594952 [Accessed 2 Sep. 2024].

Pilot, D. (2023). 7 Techniques to Handle Class Imbalance in Machine Learning. [online] Medium. Available at: https://medium.com/@data.pilot/7-techniques-to-handle-class-imbalance-in-machine-learning-eb1297419ec9 [Accessed 1 Sep. 2024].

Python, R. (n.d.). The Pandas DataFrame: Make Working With Data Delightful – Real Python. [online] realpython.com. Available at: https://realpython.com/pandas-dataframe/.

Sayedi, H. (2021). What is Sentiment Analysis? Types and Use Cases. [online] www.taus.net. Available at: https://www.taus.net/resources/blog/what-is-sentiment-analysis-types-and-use-cases.

scikit-learn.org. (n.d.). sklearn.metrics.precision_score — scikit-learn 0.23.2 documentation. [online] Available at: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html.

scikit-learn.org2. (n.d.). sklearn.metrics.recall_score — scikit-learn 0.23.2 documentation. [online] Available at: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html.

Singh, P., Singh, N., Singh, K.K. and Singh, A. (2021). Chapter 5 - Diagnosing of disease using machine learning. [online] ScienceDirect. Available at: https://www.sciencedirect.com/science/article/abs/pii/B9780128212295000033.

TechLatest.net (2023). Running Open Source LLMs in Jupyter Python Notebook as an Alternative to ChatGPT | Medium. [online] Medium. Available at: https://medium.com/@techlatest.net/running-open-source-llms-in-jupyter-python-notebook-as-an-alternative-to-chatgpt-step-by-step-9e9db50b1198 [Accessed 1 Sep. 2024].

Understand.ai (n.d.). Significance of Ground Truth Labels in Machine Learning / understand.ai. [online] understand.ai. Available at: https://understand.ai/blog/significance-of-ground-truth-labels-in-machine-learning.

Ved, M., (2020). Text Preprocessing For NLP and Machine Learning Tasks - Experfy Insights. [online] Available at: https://resources.experfy.com/ai-ml/text-preprocessing-for-nlp-and-machine-learning-tasks/ [Accessed 23 Jul. 2024].

Zakaria, A. (2024). Tokenization: The Cornerstone for NLP Tasks. [online] Machine Learning Archive. Available at: https://mlarchive.com/natural-language-processing/tokenization-the-cornerstone-for-nlp-tasks/ [Accessed 1 Sep. 2024].